In [ ]:
! wget https://classes.engr.oregonstate.edu/eecs/fall2023/ai534-400/unit1/hw1/hw1-data.tgz

In [ ]:
! tar -xzvf hw1-data.tgz

In [3]:
import numpy as np

class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        # Compute the distance based on the chosen metric
        if self.distance_metric == 'euclidean':
            distances = np.linalg.norm(self.X_train - x, axis=1)
        elif self.distance_metric == 'manhattan':
            distances = np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError("Invalid distance metric")

        # Use np.argpartition to get indices of the k smallest distances
        k_indices = np.argpartition(distances, self.k)[:self.k]

        # Extract the labels of the k nearest neighbor training samples
        k_nearest_labels = self.y_train[k_indices]

        # Return the most common class label using NumPy
        labels, counts = np.unique(k_nearest_labels, return_counts=True)
        most_common_label = labels[np.argmax(counts)]

        return most_common_label

    def kneighbors(self, X, return_distance=True):
        neighbors = []

        for x in X:
            # Compute the distance based on the chosen metric
            if self.distance_metric == 'euclidean':
                distances = np.linalg.norm(self.X_train - x, axis=1)
            elif self.distance_metric == 'manhattan':
                distances = np.sum(np.abs(self.X_train - x), axis=1)
            else:
                raise ValueError("Invalid distance metric")

            # Use np.argpartition to get indices of the k smallest distances
            k_indices = np.argpartition(distances, self.k)[:self.k]

            if return_distance:
                k_distances = distances[k_indices]
                neighbors.append((k_distances, k_indices))
            else:
                neighbors.append(k_indices)

        return neighbors


In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder

In [5]:
columns=["age","sector","edu","marriage","occupation","race","sex","hours","country","target"]
dev_data = pd.read_csv("hw1-data/income.dev.txt", sep="," , names =columns )
train_data = pd.read_csv("hw1-data/income.train.txt.5k", sep=",", names=columns)
dev_data_features = dev_data.drop('target', axis=1)
train_data_features = train_data.drop('target', axis=1)
dev_data_label = dev_data['target']
train_data_label = train_data['target']
num_processor=MinMaxScaler(feature_range=(0,2))
cat_processor=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
preprocessor=ColumnTransformer([ ('num',num_processor,['age','hours']), ('cat',cat_processor,["sector","edu","marriage","occupation","race","sex","country"]) ])
preprocessor.fit(train_data_features)
binary_train_data_features=preprocessor.transform(train_data_features)
binary_dev_data_features= preprocessor.transform(dev_data_features)
best_dev_error_rate = float('inf')
best_k = None
for k in [3]:

  knn = KNN(k=k, distance_metric='manhattan')
  knn.fit(binary_train_data_features, train_data_label)


  train_pred = knn.predict(binary_train_data_features)
  dev_pred = knn.predict(binary_dev_data_features)

  train_positive_count = sum(train_pred == " >50K")
  train_positive_rate = train_positive_count / len(train_pred) * 100

  dev_positive_count = sum(dev_pred == " >50K")
  dev_positive_rate = dev_positive_count / len(dev_pred) * 100

  train_err = (1 - accuracy_score(train_data_label, train_pred)) * 100
  dev_err = (1 - accuracy_score(dev_data_label, dev_pred)) * 100


  if dev_err < best_dev_error_rate:
    best_dev_error_rate = dev_err
    best_k = k


  print(f"k={k} train_err: {train_err:.2f}% (+: {train_positive_rate:.2f}%) dev_err: {dev_err:.2f}% (+: {dev_positive_rate:.2f}%)")
print(f"\nBest development error rate: {best_dev_error_rate:.2f}% for k={best_k}")

k=3 train_err: 11.64% (+: 23.94%) dev_err: 19.70% (+: 26.10%)

Best development error rate: 19.70% for k=3


In [6]:
len(preprocessor.get_feature_names_out())

92

In [ ]:
preprocessor.get_feature_names_out()

distance verification using  manhattan distance

In [8]:
! head -1 'hw1-data/income.dev.txt'
selected_row=binary_dev_data_features[0]
neighbors  = knn.kneighbors([selected_row])
distances, indices = neighbors[0]
distances
indices


45, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 45, United-States, <=50K


array([1084, 4872, 4787])

In [10]:
! sed -n 4873p 'hw1-data/income.train.txt.5k'
! sed -n 4788p 'hw1-data/income.train.txt.5k'
! sed -n 1085p 'hw1-data/income.train.txt.5k'

33, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 42, United-States, >50K
47, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 45, Germany, >50K
45, Federal-gov, Bachelors, Married-civ-spouse, Exec-managerial, White, Male, 40, United-States, <=50K


distance verification using  Euclidean distance

In [11]:
for k in [3]:

  knn = KNN(k=k, distance_metric='euclidean')
  knn.fit(binary_train_data_features, train_data_label)


  train_pred = knn.predict(binary_train_data_features)
  dev_pred = knn.predict(binary_dev_data_features)

  train_positive_count = sum(train_pred == " >50K")
  train_positive_rate = train_positive_count / len(train_pred) * 100

  dev_positive_count = sum(dev_pred == " >50K")
  dev_positive_rate = dev_positive_count / len(dev_pred) * 100

  train_err = (1 - accuracy_score(train_data_label, train_pred)) * 100
  dev_err = (1 - accuracy_score(dev_data_label, dev_pred)) * 100



  if dev_err < best_dev_error_rate:
    best_dev_error_rate = dev_err
    best_k = k


  print(f"k={k} train_err: {train_err:.2f}% (+: {train_positive_rate:.2f}%) dev_err: {dev_err:.2f}% (+: {dev_positive_rate:.2f}%)")
print(f"\nBest development error rate: {best_dev_error_rate:.2f}% for k={best_k}")

k=3 train_err: 11.40% (+: 23.90%) dev_err: 19.60% (+: 25.40%)

Best development error rate: 19.60% for k=3


In [12]:
neighbors  = knn.kneighbors([selected_row])
distances, indices = neighbors[0]
distances
indices

array([4787, 4872, 2591])

In [13]:
! sed -n 4873p 'hw1-data/income.train.txt.5k'
! sed -n 4788p 'hw1-data/income.train.txt.5k'
! sed -n 2592p 'hw1-data/income.train.txt.5k'

33, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 42, United-States, >50K
47, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 45, Germany, >50K
48, Federal-gov, Bachelors, Married-civ-spouse, Prof-specialty, White, Male, 44, United-States, >50K


Distance verification using KNN Classifier

In [ ]:
for k in [3]:

  knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
  knn.fit(binary_train_data_features, train_data_label)


  train_pred = knn.predict(binary_train_data_features)
  dev_pred = knn.predict(binary_dev_data_features)

  train_positive_count = sum(train_pred == " >50K")
  train_positive_rate = train_positive_count / len(train_pred) * 100

  dev_positive_count = sum(dev_pred == " >50K")
  dev_positive_rate = dev_positive_count / len(dev_pred) * 100

  train_err = (1 - accuracy_score(train_data_label, train_pred)) * 100
  dev_err = (1 - accuracy_score(dev_data_label, dev_pred)) * 100



  if dev_err < best_dev_error_rate:
    best_dev_error_rate = dev_err
    best_k = k


  print(f"k={k} train_err: {train_err:.2f}% (+: {train_positive_rate:.2f}%) dev_err: {dev_err:.2f}% (+: {dev_positive_rate:.2f}%)")
print(f"\nBest development error rate: {best_dev_error_rate:.2f}% for k={best_k}")

In [22]:
! head -1 'hw1-data/income.dev.txt'
selected_row=binary_dev_data_features[0]
neighbors  = knn.kneighbors([selected_row])
neighbors

45, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 45, United-States, <=50K


(array([[0.33441929, 1.41527469, 1.41674697]]), array([[4872, 4787, 2591]]))

In [23]:
! sed -n 4873p 'hw1-data/income.train.txt.5k'
! sed -n 4788p 'hw1-data/income.train.txt.5k'
! sed -n 2592p 'hw1-data/income.train.txt.5k'

33, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 42, United-States, >50K
47, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 45, Germany, >50K
48, Federal-gov, Bachelors, Married-civ-spouse, Prof-specialty, White, Male, 44, United-States, >50K


Distance verification for manhattan

In [24]:
for k in [3]:

  knn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
  knn.fit(binary_train_data_features, train_data_label)


  train_pred = knn.predict(binary_train_data_features)
  dev_pred = knn.predict(binary_dev_data_features)

  train_positive_count = sum(train_pred == " >50K")
  train_positive_rate = train_positive_count / len(train_pred) * 100

  dev_positive_count = sum(dev_pred == " >50K")
  dev_positive_rate = dev_positive_count / len(dev_pred) * 100

  train_err = (1 - accuracy_score(train_data_label, train_pred)) * 100
  dev_err = (1 - accuracy_score(dev_data_label, dev_pred)) * 100



  if dev_err < best_dev_error_rate:
    best_dev_error_rate = dev_err
    best_k = k


  print(f"k={k} train_err: {train_err:.2f}% (+: {train_positive_rate:.2f}%) dev_err: {dev_err:.2f}% (+: {dev_positive_rate:.2f}%)")
print(f"\nBest development error rate: {best_dev_error_rate:.2f}% for k={best_k}")

k=3 train_err: 11.68% (+: 23.98%) dev_err: 20.10% (+: 26.50%)

Best development error rate: 19.30% for k=3


In [25]:
! head -1 'hw1-data/income.dev.txt'
selected_row=binary_dev_data_features[0]
neighbors  = knn.kneighbors([selected_row])
neighbors

45, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 45, United-States, <=50K


(array([[0.38999161, 2.05479452, 2.10204082]]), array([[4872, 4787, 1084]]))

In [26]:
! sed -n 4873p 'hw1-data/income.train.txt.5k'
! sed -n 4788p 'hw1-data/income.train.txt.5k'
! sed -n 1085p 'hw1-data/income.train.txt.5k'

33, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 42, United-States, >50K
47, Federal-gov, Bachelors, Married-civ-spouse, Adm-clerical, White, Male, 45, Germany, >50K
45, Federal-gov, Bachelors, Married-civ-spouse, Exec-managerial, White, Male, 40, United-States, <=50K
